# Setup

In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [2]:
import dice
import json
import warnings

from dice.repo import Repository
from dice.query import protocol_fingerprint

# Fill repository

In [3]:
warnings.simplefilter(action='ignore', category=UserWarning)

In [4]:
sample_lab = dice.new_label("sample", "simple-label")

def cls_init(r: Repository) -> None:
    r.add_labels(sample_lab)

def cls_handler(r: Repository) -> None:
    fps = r.get_fingerprints()
    fp_labs = []
    for _, fp in fps.iterrows():
        fp_labs.append(dice.new_fp_label(fp["id"], sample_lab.id))
    r.label(*fp_labs)

def fp_handler(r: Repository) -> None:
    df = r.get_records()
    df["data"] = df["data"].apply(json.loads)

    fps = []
    for _, d in df.iterrows():
        pd = d["data"]["sample"]
        rs = {
            "vendor": pd["vendor"],
            "product": pd["product"],
            "version": pd["version"]
        }
        fp = dice.new_fingerprint(
            "sample", 
            d["ip"], 
            d["id"], 
            json.dumps(rs), 
            port=d["port"], 
            protocol="sample"
        )
        
        fps.append(fp)

    r.fingerprint(*fps)
    return

In [5]:
sample_cls = dice.new_classifier(cls_init, cls_handler, preffix="cls")
sample_fp = dice.new_fingerprinter(fp_handler, preffix="fp")

In [6]:
srcs = dice.load_sources("sources", name="zgrab2")
engine = dice.new_engine(sample_cls, sample_fp)
repo = engine.run(srcs)

# Plots

In [10]:
from plots.sankey import sankey_figure, make_lab_conf

lconfs = [
    make_lab_conf("*", 'red'),
    make_lab_conf("*:v1.0", 'rgba(100,100,100,0.5)'),
    make_lab_conf("*:prod-2", 'rgba(0,0,0,0.7)'),
]

s_repo = repo.with_view("sample", protocol_fingerprint(repo, "sample", protocol="sample"))
fig = sankey_figure(s_repo, "vendor", "product", "version", lconfs=lconfs, prefix="data")
fig.show()